In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.58 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  35.83 MB/s


In [2]:
#Gather UK average house price data 
df = pd.read_csv("http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Average-prices-2019-01.csv")
#df = pd.read_csv("http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/UK-HPI-full-file-2019-01.csv")
print(df.shape)
#Remove historical data
df= df[df['Date']==df['Date'].max()]
print(df.shape)

#remove unneeded columns
df.drop(columns=['Date','Monthly_Change','Annual_Change','Average_Price_SA'], inplace=True)
df.dropna(inplace=True)
print(df.shape)

#Exclude not London boroughs
#Area code - Code of geography (Country, Regional, County/Unitary/District Authority and London Borough)
included_areas = ['E09']
for area in included_areas:
    df = df[df['Area_Code'].str.slice(stop=3)==area]

print(df.shape)
df = df[(df['Region_Name']=="City of Westminster")|(df['Region_Name']=="City of London")]
#df = df[(df['Region_Name']=="City of Westminster")]
print(df.shape)

(125874, 7)
(441, 7)
(441, 3)
(33, 3)
(2, 3)


In [3]:
#Add lontitude and lattitude data
addresses = df['Region_Name']
latitude = []
longitude = []

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", London, UK")
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(0)
        longitude.append(0)
        
df['Latitude']= latitude
df['Longitude']= longitude

In [4]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df.head()

2


,Region_Name,Area_Code,Average_Price,Latitude,Longitude
125514,City of Westminster,E09000033,959466.0747,51.497321,-0.137149
125524,City of London,E09000001,868257.9880,51.515618,-0.091998


In [5]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
categoryId = '4bf58dd8d48988d103951735'

In [82]:
url1 = 'https://api.foursquare.com/v2/venues/search?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605&ll=51.4973206,-0.137149&radius=1000&limit=100&categoryId=4bf58dd8d48988d103951735'
results1 = requests.get(url1).json()["response"]

results2 = json_normalize(results1['venues'])
resultsCat = json_normalize(data=results1['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
df3 = pd.concat([results2, resultsCat], axis=1)
df3 = df3[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
df3.head()


,id,name,location.distance,location.formattedAddress,location.lat,location.lng,cat_name
0,50c09588e4b093aa4bfa9b8a,Gap,62,[United Kingdom],51.497057,-0.136353,Clothing Store
1,5623e425498ee6007e715dc8,Mango,275,"[70 Victoria Street, London, Greater London, S...",51.496900,-0.141074,Women's Store
2,4c38a7ffdfb0e21ebff1aea8,Zara,184,"[82 Victoria St, London, Greater London, SW1E ...",51.496998,-0.139760,Clothing Store
3,4cd952f65e1b721e57843ed9,Topshop,208,"[Unit 18 Cardinal Place, London, Greater Londo...",51.496944,-0.140094,Women's Store
4,4b7bf0b6f964a520e0742fe3,Marks & Spencer,308,"[Cardinal Place (Victoria St), London, Greater...",51.497774,-0.141537,Clothing Store


In [213]:
#What are the most popular places nearby?

def getNearbyVenues(names, latitudes, longitudes, avg_price, radius=1000):
    
    
    venue_list = []
    df5 = pd.DataFrame()
    
    for name, lat, lng, price in zip(names, latitudes, longitudes, avg_price):
                    
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
            
        # make the GET request
        print(url)
        
        results = requests.get(url).json()["response"]
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results1['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        df3 = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        for i in range(0,resultsVen.shape[0]):
            venue_list.append(name)
        
        venues_list = pd.DataFrame(venue_list, columns=['Name'])
        df4 = pd.concat([venues_list, df3], axis=1)
         
        

    
    
    return(df4)

In [217]:
df['Region_Name'].unique
#loop through Getnearbyvalues with each region name.

<bound method Series.unique of 125514    City of Westminster
125524         City of London
Name: Region_Name, dtype: object>

In [ ]:
venue_ids = getNearbyVenues(names=df['Region_Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   avg_price=df['Average_Price']
                                  )
venue_ids
#venue_ids.rename(columns={'id':'Venue ID','name':'Venue Name'})


In [130]:
for id in venue_ids:                    
        # create the API request URL
        #client_id={}&client_secret={}
        url1 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(id,CLIENT_ID,CLIENT_SECRET,VERSION)
        print(url1)
        
        #r1 = requests.get(url1)
        
        #if 'ids' in r1.json()['response']:
         #   ids = r1.json()['response']
#
 #           for venue in venues:
  #                  venue_ids.add(venue['id'])
   #                 venue_ids.add(venue['location']['lat'])
    #                venue_ids.add(venue['location']['lng']) 

https://api.foursquare.com/v2/venues/4fa8f790e4b0396559447c83?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605
https://api.foursquare.com/v2/venues/4da47beab3e7236aee8f4779?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605
https://api.foursquare.com/v2/venues/532015ab498e96a0380c1ad2?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605
https://api.foursquare.com/v2/venues/5318bd18498e34719ed0445f?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605
https://api.foursquare.com/v2/venues/53205748498ea0692dc1fb75?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605
https://api.foursqua

In [115]:
london_venues
#london_venues[(london_venues['Region Name']=="City of Westminster")&(london_venues['Venue Category']=="Clothing Store")]
#london_venues['Venue Category'].unique
#london_venues[['Region Name','Average Price']].groupby('Region Name').min()

{-0.1378011703491211,
 -0.13717658711982494,
 '4fa8f790e4b0396559447c83',
 -0.13672828674316406,
 -0.13684630393981934,
 -0.1371520757675171,
 -0.13644933700561523,
 -0.13811230659484863,
 -0.13760805130004883,
 -0.13657808303833008,
 -0.13728953897953033,
 -0.1355588436126709,
 '532015ab498e96a0380c1ad2',
 -0.09364128112792969,
 -0.09243965148925781,
 -0.09138286113739014,
 51.516213,
 '4d638eddbd2537042658cec5',
 -0.09203195571899414,
 -0.0925469398498535,
 -0.09055580198764801,
 -0.09246110916137695,
 -0.0922572612762451,
 '4b6a66adf964a52087d42be3',
 51.497495327145884,
 51.49737658280867,
 51.49832004231382,
 51.49640555192821,
 51.496401,
 51.497616,
 51.497706,
 51.51555072911845,
 51.51501535470732,
 51.514984,
 -0.13790728067565713,
 -0.13761714638224995,
 -0.0905658722522302,
 -0.093388,
 -0.09169688864926237,
 '4d9344837b5ea1437743c4b8',
 51.4972487534507,
 51.51628805825127,
 '588b664cdc332914bc023aba',
 -0.091825,
 '4cbc0d44f50e224bf5ebfcfb',
 '4bb0ba5ff964a52003583ce3',
 

In [ ]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe

london_onehot['Average Price'] = london_venues['Average Price'] 
london_onehot['Region Name'] = london_venues['Region Name'] 
fixed_columns = [london_onehot.columns[-1]] + [london_onehot.columns[-2]] + list(london_onehot.columns[:-2])
london_onehot = london_onehot[fixed_columns]

df1 = london_onehot.groupby(['Region Name','Average Price']).sum()
df1.reset_index()
